In [1]:
import torch
import torchaudio
import torch.optim as optim
from torch import nn
from torchinfo import summary
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from model import Denoiser
from audio_dataset import Audio_Dataset
from trainer import Trainer
from loss import DenoiserLoss


In [2]:
config = {
    "device" : torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir" : "data",
    "batch_size" : 32,
    "epochs" : 10,
    "learning_rate": 0.00025,
    "batches_per_epoch": 1105,
    "batches_per_epoch_val": 315,
    "train": 0.7,
    "val": 0.2,
}

In [3]:
train_dataset = Audio_Dataset(config, set_type="train")
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= config["batch_size"],
    shuffle=True,
    drop_last=True,
    num_workers=1,
)

val_dataset = Audio_Dataset(config, set_type="val")
val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    drop_last=True,
    num_workers=1,
)

In [4]:
trained_weights = './model_final'

model = Denoiser(depth=5, N_attention=1)
# model.load_state_dict(torch.load(trained_weights))
model.to(config["device"])
print(model)

loss_fn = DenoiserLoss()
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, factor=0.5, patience=20, verbose=True, threshold=0.00001
)

Denoiser(
  (encoder): ModuleList(
    (0): Sequential(
      (0): Conv1d(1, 48, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(48, 96, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (1): Sequential(
      (0): Conv1d(48, 96, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(96, 192, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (2): Sequential(
      (0): Conv1d(96, 192, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(192, 384, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (3): Sequential(
      (0): Conv1d(192, 384, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(384, 768, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (4): Sequential(
      (0): Conv1d(384, 768, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(768, 1536, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
  )
  (attention): ModuleList(
    (0): Sequenti

In [5]:
summary(model)

Layer (type:depth-idx)                                       Param #
Denoiser                                                     --
├─ModuleList: 1-1                                            --
│    └─Sequential: 2-1                                       --
│    │    └─Conv1d: 3-1                                      432
│    │    └─ReLU: 3-2                                        --
│    │    └─Conv1d: 3-3                                      4,704
│    │    └─GLU: 3-4                                         --
│    └─Sequential: 2-2                                       --
│    │    └─Conv1d: 3-5                                      36,960
│    │    └─ReLU: 3-6                                        --
│    │    └─Conv1d: 3-7                                      18,624
│    │    └─GLU: 3-8                                         --
│    └─Sequential: 2-3                                       --
│    │    └─Conv1d: 3-9                                      147,648
│    │    └─ReLU: 

In [6]:
trainer = Trainer(
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=config["epochs"],
    config=config,
    scheduler=scheduler,
)

model = trainer.train(train_dataloader,val_dataloader)

100%|██████████| 1106/1106 [08:13<00:00,  2.24it/s]


Training epoch complete


100%|██████████| 316/316 [02:16<00:00,  2.32it/s]


Eval epoch complete
Epoch: 1/10, Train Loss=3.4527990818, Val Loss=1.7036632139


100%|██████████| 1106/1106 [08:25<00:00,  2.19it/s]


...

Eval epoch complete
Epoch: 9/10, Train Loss=0.5243663192, Val Loss=0.7608897661


100%|██████████| 1106/1106 [08:20<00:00,  2.21it/s]


Training epoch complete


100%|██████████| 316/316 [02:14<00:00,  2.35it/s]

Eval epoch complete
Epoch: 10/10, Train Loss=0.417563796, Val Loss=0.422070321
